In [1]:
import pyspark
from pyspark.sql import *
from pyspark.sql import SQLContext

from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext

from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.functions import udf, col
from pyspark.ml.evaluation import RegressionEvaluator

spark = SparkSession.builder.master("local[2]").config("spark.driver.host","localhost").appName("Titanic_survival").getOrCreate()
sc = spark.sparkContext

# using SQLContext to read parquet file
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

spark

In [2]:
df = spark.read.csv('titanic.csv', inferSchema = True, header = True)
df.show(3)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
only showing top 3 rows



In [3]:
df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [4]:
df.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [5]:
req_cols = df.select(['Survived','Pclass','Sex','Age','SibSp','Parch','Fare','Embarked'])

In [6]:
final_data = req_cols.na.drop()

In [7]:
from pyspark.ml.feature import VectorAssembler,StringIndexer,VectorIndexer,OneHotEncoder
gender_indexer = StringIndexer(inputCol = 'Sex', outputCol = 'SexIndex')
gender_encoder = OneHotEncoder(inputCol='SexIndex', outputCol = 'SexVec')

In [8]:
embark_indexer = StringIndexer(inputCol = 'Embarked', outputCol = 'EmbarkIndex')
embark_encoder = OneHotEncoder(inputCol = 'EmbarkIndex', outputCol = 'EmbarkVec')

In [9]:
assembler = VectorAssembler(inputCols = ['Pclass', 'SexVec', 'Age', 'SibSp', 'Parch', 'Fare', 'EmbarkVec'], outputCol = 'features')

In [10]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
log_reg=LogisticRegression(featuresCol='features',labelCol='Survived')

In [11]:
log_reg

LogisticRegression_d8a3bc4bd5f5

In [12]:
pipeline = Pipeline(stages = [gender_indexer, embark_indexer, 
                             gender_encoder, embark_encoder,
                             assembler, log_reg])

In [13]:
train, test = final_data.randomSplit([0.7, 0.3])

In [14]:
fit_model = pipeline.fit(train)

In [15]:
results = fit_model.transform(test)

In [16]:
results.show()

+--------+------+------+----+-----+-----+-------+--------+--------+-----------+-------------+-------------+--------------------+--------------------+--------------------+----------+
|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|Embarked|SexIndex|EmbarkIndex|       SexVec|    EmbarkVec|            features|       rawPrediction|         probability|prediction|
+--------+------+------+----+-----+-----+-------+--------+--------+-----------+-------------+-------------+--------------------+--------------------+--------------------+----------+
|       0|     1|female|25.0|    1|    2| 151.55|       S|     1.0|        0.0|    (1,[],[])|(2,[0],[1.0])|[1.0,0.0,25.0,1.0...|[-2.6669525038145...|[0.06495180720228...|       1.0|
|       0|     1|  male|28.0|    0|    0|   47.1|       S|     0.0|        0.0|(1,[0],[1.0])|(2,[0],[1.0])|[1.0,1.0,28.0,0.0...|[-0.4444117968093...|[0.39069022994308...|       1.0|
|       0|     1|  male|36.0|    0|    0| 40.125|       C|     0.0|        1.0|(1,[0],[1.0

In [17]:
results.select('rawPrediction','prediction', 'Survived').show()

+--------------------+----------+--------+
|       rawPrediction|prediction|Survived|
+--------------------+----------+--------+
|[-2.6669525038145...|       1.0|       0|
|[-0.4444117968093...|       1.0|       0|
|[-0.1109118863112...|       1.0|       0|
|[0.30623370086394...|       0.0|       0|
|[0.07534622253561...|       0.0|       0|
|[0.12175704686009...|       0.0|       0|
|[0.64163150673276...|       0.0|       0|
|[0.38975397357102...|       0.0|       0|
|[0.71439031422145...|       0.0|       0|
|[0.87937160060006...|       0.0|       0|
|[0.82844542541837...|       0.0|       0|
|[0.92244363557563...|       0.0|       0|
|[1.11675231018841...|       0.0|       0|
|[1.25130803334571...|       0.0|       0|
|[1.89743315975321...|       0.0|       0|
|[-1.1986309035008...|       1.0|       0|
|[0.46791471512801...|       0.0|       0|
|[0.46791471512801...|       0.0|       0|
|[0.39466023278806...|       0.0|       0|
|[0.51550706336121...|       0.0|       0|
+----------

In [18]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

eval = BinaryClassificationEvaluator(rawPredictionCol = 'rawPrediction', labelCol = 'Survived')
AUC = eval.evaluate(results)
AUC

0.8527899447724242